# Fine-tuning

En este notebook, ajustamos un sentencetransformers embedding model de código abierto con nuestro conjunto de datos generado sintéticamente.

### Cargar modelo pre-entrenado

In [1]:
from sentence_transformers import SentenceTransformer

In [2]:
model_id = "BAAI/bge-small-en"
model = SentenceTransformer(model_id)

In [3]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

### Definir dataloader

In [4]:
import json

from torch.utils.data import DataLoader
from sentence_transformers import InputExample

In [5]:
TRAIN_DATASET_FPATH = './data/train_dataset.json'
VAL_DATASET_FPATH = './data/val_dataset.json'

# usamos un tamaño de lote muy pequeño para ejecutar este ejemplo de prueba en una máquina local
# normalmente debería ser mucho más grande

BATCH_SIZE = 15

In [6]:
with open(TRAIN_DATASET_FPATH, 'r+') as f:
    train_dataset = json.load(f)

with open(VAL_DATASET_FPATH, 'r+') as f:
    val_dataset = json.load(f)

In [7]:
dataset = train_dataset

corpus = dataset['corpus']
queries = dataset['queries']
relevant_docs = dataset['relevant_docs']

examples = []
for query_id, query in queries.items():
    try:
        if query_id in relevant_docs:
            node_id = relevant_docs[query_id][0]
            text = corpus[node_id]
            example = InputExample(texts=[query, text])
            examples.append(example)
        else:
            print(f"Query ID {query_id} not found in relevant_docs dictionary.")
    except KeyError:
        print(f"Query ID {query_id} not found in relevant_docs dictionary.")


In [8]:
loader = DataLoader(
    examples, batch_size = BATCH_SIZE
)

### Definir pérdida

**MultipleNegativesRankingLoss** es una gran función de pérdida si solo tiene pares positivos, por ejemplo, solo pares de textos similares como pares de paráfrasis, pares de preguntas duplicadas, pares de (consulta, respuesta) o pares de (idioma_origen, idioma_destino).

Esta función de pérdida funciona muy bien para entrenar modelos de embeddings para configuraciones RAG en las que tiene pares positivos (por ejemplo, (consulta, documento_relevante)), ya que tomará muestras en cada lote de n-1 documentos negativos de forma aleatoria.

El rendimiento suele aumentar al aumentar el tamaño de los lotes.

Para obtener más detalles, consulte:
* [docs](https://www.sbert.net/docs/package_reference/losses.html)
* [paper]()

In [9]:
from sentence_transformers import losses

In [10]:
loss = losses.MultipleNegativesRankingLoss(model)

### Definir evaluador

Configuramos un evaluador con nuestra división de valores del conjunto de datos para monitorear qué tan bien se está desempeñando el modelo de incorporación durante el entrenamiento.

In [11]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

In [12]:
dataset = val_dataset

corpus = dataset['corpus']
queries = dataset['queries']
relevant_docs = dataset['relevant_docs']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

### Ejecutar entrenamiento

El ciclo de entrenamiento es muy sencillo de intensificar gracias a la API de entrenamiento de modelos de alto nivel de SentenceTransformers.
Todo lo que tenemos que hacer es conectar el cargador de datos, la función de pérdida y el evaluador que definimos en las celdas anteriores (junto con un par de configuraciones menores adicionales).

In [13]:
# entrenamos el modelo durante muy pocas epochs en este ejemplo de prueba
# normalmente este valor debería ser mayor para obtener un mejor rendimiento

EPOCHS = 3

In [14]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives = [(loader, loss)],
    epochs = EPOCHS,
    warmup_steps = warmup_steps,
    output_path ='exp_finetune',
    show_progress_bar = True,
    evaluator = evaluator, 
    evaluation_steps = 50,
)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/56 [00:00<?, ?it/s]

Iteration:   0%|          | 0/56 [00:00<?, ?it/s]

Iteration:   0%|          | 0/56 [00:00<?, ?it/s]